In [ ]:
import xarray as xr
import pandas as pd
import numpy as np
import geopandas as gp
from os.path import join, basename
from datetime import date, datetime
import os

In [ ]:
import matplotlib.pyplot as plt
from plot_tools import *
import cartopy.crs as ccrs
import seaborn as sns

In [ ]:
root = r'/scratch/compound_hotspots'
ddir = join(root, 'data', '4-postprocessed')
fdir = join(root, 'reports', 'figures')

In [ ]:
rc = {'savefig.bbox': 'tight',  'savefig.format': 'png', 'savefig.dpi':300}
context = 'paper'
# sns.set(context=context, style='whitegrid', font_scale=0.75 if context == 'talk' else 1., rc=rc)
sns.set(context=context, style='whitegrid', font_scale=1.2 if context == 'paper' else 1., rc=rc)

crs = ccrs.Robinson()
crs_sub = ccrs.PlateCarree()

cmap_div = sns.diverging_palette(220, 10, s=75, l=40, sep=1, as_cmap=True)
bmap_kwargs = dict()
#     features=['land', 'rivers'],
#     feat_colors = [cfeature.COLORS['land_alt1'], cfeature.COLORS['water']],
# )

plot_kwargs=dict(edgecolor=(0.5, 0.5, 0.5, 0.8), linewidth=0.5, legend=False, zorder=2)
box_kwargs=dict(whis=[5,95], showfliers=False, boxprops=dict(linewidth=1.5), medianprops=dict(linewidth=1.5))

regions = {
    'A': (-100.0, 22.0, -50.0, 53.0), # 30 x 18
    'B': (-12.0, 35.0, 38.0, 65.0), # 50 x 30
    'C': (100.0, 20.0, 150.0, 50.0),  # 50 x 30
}
from string import ascii_uppercase as letters

In [ ]:
attrs_fn = join(ddir, 'rivmth_mean_attrs.csv')
attrs = pd.read_csv(attrs_fn, index_col='index').rename(columns={'rivmth_lat':'lat', 'rivmth_lon':'lon'})
attrs['Q_mean_log10'] = np.log10(attrs['Q_mean'])
attrs['markersize'] = np.maximum((attrs['Q_mean_log10'] / attrs['Q_mean_log10'].max()),0) * 30 + 15


## rp peak ratio

In [ ]:
from scipy.stats import pearsonr, zscore
import pandas as pd
def nanzscore(x):
    z = np.ones_like(x)
    valid = np.isnan(x)==False
    z[valid] = zscore(x[valid])
    return z 

In [ ]:
q = 95
min_dist = 14
alpha=0.25
method='_empirical'
# fn_peaks_rp_ci = join(ddir, f'rivmth_peaks_q{q}d{min_dist}_rp_ci.nc')
fn_peaks_rp = join(ddir, f'rivmth_peaks_q{q}d{min_dist}_rp.nc')
da_wse_rps = xr.open_dataset(fn_peaks_rp)['WSE_peaks']

# diff
diff_surge_seas = (da_wse_rps.sel(scen='surge') - da_wse_rps.sel(scen='seas'))
diff_surge_seas.name = 'diff_surge_seas'
diff_seas_tide = (da_wse_rps.sel(scen='seas') - da_wse_rps.sel(scen='tide'))
diff_seas_tide.name = 'diff_seas_tide'
diff_surge_tide = (da_wse_rps.sel(scen='surge') - da_wse_rps.sel(scen='tide'))
diff_surge_tide.name = 'diff_surge_tide'
ds_diff = xr.merge([
    diff_surge_seas,
    diff_seas_tide,
    diff_surge_tide
])

dim = 'ensemble'
# average and calculate significance based on std error
N = ds_diff[dim].size
ds_diff_mean = ds_diff.mean(dim)
ds_diff_dir =  xr.ufuncs.fabs(xr.ufuncs.sign(ds_diff).sum(dim)) == N
ds_diff_sign = xr.ufuncs.fabs(ds_diff_mean / ds_diff.std(dim)) > (2 / xr.ufuncs.sqrt(N-1))

# ds_lst = []
# for var in list(ds_diff.data_vars.keys()):
#     v1,v2,v3 = var.split('_')
#     da_sign = xr.where(
#         xr.ufuncs.sign(ds_diff[var])>1,
#         da_wse_rps_ci.sel(scen=v2, alpha=alpha) > da_wse_rps.sel(scen=v3),
#         da_wse_rps_ci.sel(scen=v3, alpha=alpha) > da_wse_rps.sel(scen=v2)
#     ).drop('alpha')
#     da_sign.name = var
#     ds_lst.append(da_sign)
# ds_diff_sign2 = xr.merge(ds_lst).sum('ensemble') == 1

ds_diff_stats = xr.concat([
    ds_diff_mean,
    np.logical_and(ds_diff_sign, ds_diff_dir),
], dim='stat') #.reset_coords(drop=True) #.drop(['diff_surge_seas', 'ratio_surge_seas'])

ds_diff_stats['stat'] = xr.Variable('stat', ['mean', 'sign'])
fn_diff_stats = join(ddir, f'rivmth_peaks_q{q}d{min_dist}_rp_diff.nc')
if not os.path.isfile(fn_diff_stats):
    ds_diff_stats.to_netcdf(fn_diff_stats)
(ds_diff_stats.sel(stat='sign', rp=2)==1).sum('index')

In [ ]:
for var in ['diff_surge_tide']: #list(ds_diff_stats.data_vars.keys()):
#     if var.startswith('ratio'): continue
    for rp in [2]:
        if var.startswith('ratio'):
            label=f'ratio change return level (rp {rp} year) [-]'
            vmin, vmax, n= -0.5, 3.5, 9
        else:
            label=f'difference return level (rp {rp} year) [m]'
            vmin, vmax, n= -0.2, 0.6, 5
        gdf = pandas2geopandas(pd.concat([
            attrs[['lat', 'lon', 'markersize']],
            ds_diff_stats[var].sel(rp=rp).to_series().unstack().T
        ], axis=1))

        plt.close('all')
        fig = plt.figure(figsize=(17, 10))
        grid = plt.GridSpec(2, 6, hspace=0.1, wspace=0.4)

        column = 'mean'
        idx_sign = np.logical_and(gdf['sign']==1, np.abs(nanzscore(gdf['mean'].values))<=4)

        gdf0 = gdf[idx_sign==False].copy()
        gdf1 = gdf[idx_sign].copy().sort_values(column, ascending=True)
        df_box = ds_diff[var].sel(rp=rp, index=gdf1.index).to_series().unstack(-1).T

        cticks=np.linspace(vmin, vmax, n)
        cmap_turbo_div = ListedColormap([
            interpolate_cmap(google_turbo_data, x) for x in 
            np.hstack([np.linspace(0.55-(.4*abs(vmin)/vmax)*2, .55, int(-vmin*100)), np.linspace(0.6, 1, int(vmax*100))])
        ])
        # main map
        axg = fig.add_subplot(grid[:-1, :-1], projection=crs_sub)
        add_regions(axg, regions, dx=-6, dy=-6)
        basemap(axg, bbox=(-180, -60, 180, 90), gridlines=True, outline=True, **bmap_kwargs)
        gdf0.plot(ax=axg, marker='x', color=(0, 0, 0, 0.6), markersize=5, linewidth=0.2, alpha=0.6, legend=False)

        plot_kwargs.update(markersize=gdf1['markersize'].values, marker='o')
        cax = plot_choropleth(
            fig, axg, gdf1, column=column, 
            cmap=cmap_turbo_div, vmin=vmin, vmax=vmax, cticks=cticks, discrete=False,
            plot_kwargs=plot_kwargs,
            cbar_kwargs=dict(label=label, location='right'),
            cbar_pos = dict(pad=0.1, fraction=0.1)
        )

        # submaps 
        axs = dict()
        # gdf1 = gdf1.sort_values('markersize', ascending=True)
        for i, (name, bbox) in enumerate(regions.items()):
            ax = fig.add_subplot(grid[-1, i*2:(i+1)*2], projection=crs_sub)
            basemap(ax, bbox=bbox, gridlines=True, gridspacing=10, outline=True, **bmap_kwargs)
            gdf0.plot(ax=ax, marker='x', color=(0, 0, 0, 0.8), markersize=15, linewidth=0.2, legend=False)
            plot_kwargs.update(markersize=gdf1['markersize'].values*2, marker='o')
            plot_choropleth(
                fig, ax, gdf1, column=column, 
                cmap=cmap_turbo_div, vmin=vmin, vmax=vmax, cticks=cticks, discrete=False,
                plot_kwargs=plot_kwargs, cbar_kwargs=False
            )
            xmin, ymax = bbox[0], bbox[3]
            ax.text(xmin, ymax+1, name, transform=crs_sub)
            axs[name] = ax

        # boxplot ensemble
        box_kwargs.update(widths=0.5)
        color_props = dict(boxes="lightgrey", whiskers="k", medians="k", caps="k")
        ax = fig.add_subplot(grid[:-1, -1:])
        df_box.plot.box(ax=ax, color=color_props, patch_artist=True, **box_kwargs)
#         ax.set_ylim([cticks[1], cticks[-2]])
#         ax.yaxis.set_ticks(cticks[1:-2])
        ax.set_ylim([vmin, vmax])
        ax.yaxis.set_ticks(cticks)
        ax.yaxis.set_ticklabels([])
        ax.grid(axis='x')
        ax.set_xlabel('model')
        cpos = cax.get_position()
        bpos = ax.get_position()
        cax.set_position([cpos.x0, cpos.y0, cpos.width, cpos.height])
        # ax.set_position([cpos.x0+0.03, cpos.y0, bpos.width*1.5, cpos.height])
        ax.set_position([cpos.x0+0.03, cpos.y0+cpos.height*0.05, bpos.width*1.5, cpos.height/1.10])
        cax.yaxis.set_ticks_position('left')
        cax.yaxis.set_label_position('left')

        fn_fig = join(fdir, f'{var}_q{q}d{min_dist}_rp{rp:02d}{method}.png')
        print(basename(fn_fig))
        plt.savefig(fn_fig)
#         plt.close('all')
#         break
#     break

In [ ]:
plt.close('all')
fig = plt.figure(figsize=(17, 12))
grid = plt.GridSpec(2, 2, hspace=0.0, wspace=0.1)
rp=2
label=f'change in riverine water level \n{rp}-yr return period [m]'
vmin, vmax, n= -0.2, 0.6, 5
cticks=np.linspace(vmin, vmax, n)
cmap_turbo_div = ListedColormap([
    interpolate_cmap(google_turbo_data, x) for x in 
    np.hstack([np.linspace(0.55-(.4*abs(vmin)/vmax)*2, .55, int(-vmin*100)), np.linspace(0.6, 1, int(vmax*100))])
])

var = 'diff_surge_tide'
column='mean'
gdf = pandas2geopandas(pd.concat([
    attrs[['lat', 'lon', 'markersize']],
    ds_diff_stats[var].sel(rp=rp).to_series().unstack().T
], axis=1))
axg = fig.add_subplot(grid[0,:], projection=crs_sub)
gdf = pandas2geopandas(pd.concat([
    attrs[['lat', 'lon', 'markersize']],
    ds_diff_stats[var].sel(rp=rp).to_series().unstack().T
], axis=1))
idx_sign = np.logical_and(gdf['sign']==1, np.abs(nanzscore(gdf['mean'].values))<=4)
gdf0 = gdf[idx_sign==False].copy()
gdf1 = gdf[idx_sign].copy().sort_values('mean', ascending=True)
# main map
basemap(axg, bbox=(-180, -60, 180, 90), gridlines=True, outline=True, **bmap_kwargs)
gdf0.plot(ax=axg, marker='x', color=(0, 0, 0, 0.6), markersize=5, linewidth=0.2, alpha=0.6, legend=False)
plot_kwargs.update(markersize=gdf1['markersize'].values, marker='o')
cax = plot_choropleth(
    fig, axg, gdf1, column=column, 
    cmap=cmap_turbo_div, vmin=vmin, vmax=vmax, cticks=cticks, discrete=False,
    plot_kwargs=plot_kwargs,
    cbar_kwargs=dict(label=label, location='right'),
    cbar_pos = dict(pad=0.05, fraction=0.1)
)
axg.text(-175, 80, letters[0], transform=crs_sub, fontsize=16)

# sub map
for i, var in enumerate(['diff_surge_seas', 'diff_seas_tide']):
    axg = fig.add_subplot(grid[1,i], projection=crs_sub)
    gdf = pandas2geopandas(pd.concat([
        attrs[['lat', 'lon', 'markersize']],
        ds_diff_stats[var].sel(rp=rp).to_series().unstack().T
    ], axis=1))
    idx_sign = np.logical_and(gdf['sign']==1, np.abs(nanzscore(gdf['mean'].values))<=4)
    gdf0 = gdf[idx_sign==False].copy()
    gdf1 = gdf[idx_sign].copy().sort_values('mean', ascending=True)
    basemap(axg, bbox=(-180, -60, 180, 90), gridlines=True, outline=True, **bmap_kwargs)
    gdf0.plot(ax=axg, marker='x', color=(0, 0, 0, 0.6), markersize=5, linewidth=0.2, alpha=0.6, legend=False)
    plot_kwargs.update(markersize=gdf1['markersize'].values, marker='o')
    plot_choropleth(
        fig, axg, gdf1, column=column, 
        cmap=cmap_turbo_div, vmin=vmin, vmax=vmax, cticks=cticks, discrete=False,
        plot_kwargs=plot_kwargs, cbar_kwargs=False
    )
    axg.text(-175, 75, letters[i+1], transform=crs_sub, fontsize=16)

fig.tight_layout()
fn_fig = join(fdir, f'global_diff_q{q}d{min_dist}_rp{rp:02d}{method}.png')
print(basename(fn_fig))
plt.savefig(fn_fig)
# plt.close('all')

## timing

In [ ]:
q = 95
min_dist = 30

fn_peaks_timing = join(ddir, f'rivmth_peaks_timing.nc')
ds_timing = xr.open_dataset(fn_peaks_timing)
ds_timing_diff = ds_timing['doy_diff']


dim = 'ensemble'
# average and calculate significance based on std error
N = ds_timing_diff[dim].size
ds_timing_diff_mean = ds_timing_diff.mean(dim)
ds_timing_diff_dir =  xr.ufuncs.fabs(xr.ufuncs.sign(ds_timing_diff).sum(dim)) == N
ds_timing_diff_sign = xr.ufuncs.fabs(ds_timing_diff_mean / ds_timing_diff.std(dim)) > (2 / xr.ufuncs.sqrt(N-1))
ds_timing_sign = (ds_timing['doy_uniform_p'].max(dim).max('scen')<0.05)

ds_timing_diff_stats = xr.concat([
    ds_timing_diff_mean,
    np.logical_and(np.logical_and(ds_timing_diff_sign, ds_timing_diff_dir), ds_timing_sign),
], dim='stat')
ds_timing_diff_stats['stat'] = xr.Variable('stat', ['mean', 'sign'])

In [ ]:
import sys
import os
sys.path.append(os.path.abspath('../3-postprocess/'))

from circ_stats import circ_mean
from temp_stats import doy_to_circ, circ_to_doy

circ = doy_to_circ(ds_timing['doy_mean'], dim=None)
ds_timing_mean = circ_to_doy(circ_mean(circ, dim='ensemble'), None)
ds_timing_sign = ds_timing['doy_uniform_p'].max(dim)<0.05

ds_timing_stats = xr.concat([
    ds_timing_mean,
    ds_timing_sign,
], dim='stat')
ds_timing_stats['stat'] = xr.Variable('stat', ['mean', 'sign'])
ds_timing_stats

In [ ]:
vmin, vmax, n= -50, 50, 5
label='change in mean flood day [days]'
cticks=np.linspace(vmin, vmax, n)

gdf = pandas2geopandas(pd.concat([
    attrs[['lat', 'lon', 'markersize']],
    ds_timing_diff_stats.to_series().unstack().T
], axis=1))

plt.close('all')
fig = plt.figure(figsize=(17, 10))
grid = plt.GridSpec(2, 6, hspace=0.05, wspace=0.4)

column = 'mean'
idx_sign = gdf['sign']==1

gdf0 = gdf[idx_sign==False].copy()
gdf1 = gdf[idx_sign].copy().sort_values('markersize', ascending=True)
df_box = ds_timing_diff.sel(index=gdf1.index).to_series().unstack(-1).T


# main map
axg = fig.add_subplot(grid[:-1, :-1], projection=crs_sub)
add_regions(axg, regions, dx=-6, dy=-6)
basemap(axg, bbox=(-180, -60, 180, 90), gridlines=True, outline=True, **bmap_kwargs)
gdf0.plot(ax=axg, marker='x', color=(0, 0, 0, 0.6), markersize=5, linewidth=0.2, alpha=0.6, legend=False)
plot_kwargs.update(markersize=gdf1['markersize'].values, marker='o', linewidth=0.8)
cax = plot_choropleth(
    fig, axg, gdf1, column=column, 
    cmap=plt.cm.RdBu_r, vmin=vmin, vmax=vmax, cticks=cticks, discrete=False,
    plot_kwargs=plot_kwargs,
    cbar_kwargs=dict(label=label, location='right', extend='both'),
    cbar_pos = dict(pad=0.1, fraction=0.1)
)

# submaps 
axs = dict()
# gdf1 = gdf1.sort_values('markersize', ascending=True)
for i, (name, bbox) in enumerate(regions.items()):
    ax = fig.add_subplot(grid[-1, i*2:(i+1)*2], projection=crs_sub)
    basemap(ax, bbox=bbox, gridlines=True, gridspacing=10, outline=True, **bmap_kwargs)
    gdf0.plot(ax=ax, marker='x', color=(0, 0, 0, 0.6), markersize=15, linewidth=0.2, legend=False)
    plot_kwargs.update(markersize=gdf1['markersize'].values*2, marker='o')
    plot_choropleth(
        fig, ax, gdf1, column=column, 
        cmap=plt.cm.RdBu_r, vmin=vmin, vmax=vmax, cticks=cticks, discrete=False,
        plot_kwargs=plot_kwargs, cbar_kwargs=False
    )
    xmin, ymax = bbox[0], bbox[3]
    ax.text(xmin, ymax+1, name, transform=crs_sub)
    axs[name] = ax

# boxplot ensemble
color_props = dict(boxes="lightgrey", whiskers="k", medians="k", caps="k")
ax = fig.add_subplot(grid[:-1, -1:])
df_box.plot.box(ax=ax, color=color_props, patch_artist=True, **box_kwargs)
ax.set_ylim([vmin, vmax])
ax.yaxis.set_ticks(cticks)
ax.yaxis.set_ticklabels([])
ax.grid(axis='x')
ax.set_xlabel('model')
cpos = cax.get_position()
bpos = ax.get_position()
cax.set_position([cpos.x0, cpos.y0, cpos.width, cpos.height])
# ax.set_position([cpos.x0+0.03, cpos.y0, bpos.width*1.5, cpos.height])
ax.set_position([cpos.x0+0.03, cpos.y0+cpos.height*0.05, bpos.width*1.5, cpos.height/1.10])
cax.yaxis.set_ticks_position('left')
cax.yaxis.set_label_position('left')

fn_fig = join(fdir, f'timing_q{q}d{min_dist}_doy_diff.png')
print(basename(fn_fig))
plt.savefig(fn_fig)
# plt.close('all')


In [ ]:
amin, amax = -np.pi, np.pi
norm = mpl.colors.Normalize(-np.pi, np.pi)

# legend
dates = pd.date_range(date(2001,1,1), date(2001,12,31), freq='MS')[0::2]
dticks = dates.strftime('%b').tolist()
doys = np.array([d.timetuple().tm_yday for d in dates]).astype(float)
circ = doys/365*2*np.pi # we need to use a [0, 2pi] range here
xval = np.arange(amin, amax, 0.1)
yval = np.ones_like(xval)*2.

In [ ]:
plt.close('all')
fig = plt.figure(figsize=(17, 12))
grid = plt.GridSpec(2, 2, hspace=0.0, wspace=0.1)

# main map chnage
vmin, vmax, n= -50, 50, 5
label='change in mean annual flood day [days]'
cticks=np.linspace(vmin, vmax, n)
gdf = pandas2geopandas(pd.concat([
    attrs[['lat', 'lon', 'markersize']],
    ds_timing_diff_stats.to_series().unstack().T
], axis=1))
column = 'mean'
idx_sign = gdf['sign']==1
gdf0 = gdf[idx_sign==False].copy()
gdf1 = gdf[idx_sign].copy().sort_values('markersize', ascending=True)
axg = fig.add_subplot(grid[0, :], projection=crs_sub)
# add_regions(axg, regions, dx=-6, dy=-6)
basemap(axg, bbox=(-180, -60, 180, 90), gridlines=True, outline=True, **bmap_kwargs)
gdf0.plot(ax=axg, marker='x', color=(0, 0, 0, 0.6), markersize=5, linewidth=0.2, alpha=0.6, legend=False)
plot_kwargs.update(markersize=gdf1['markersize'].values, marker='o', linewidth=0.8)
cax = plot_choropleth(
    fig, axg, gdf1, column=column, 
    cmap=plt.cm.RdBu_r, vmin=vmin, vmax=vmax, cticks=cticks, discrete=False,
    plot_kwargs=plot_kwargs,
    cbar_kwargs=dict(label=label, location='right', extend='both'),
    cbar_pos = dict(pad=0.05, fraction=0.1)
)
axg.text(-175, 80, letters[0], transform=crs_sub, fontsize=16)

# tide
axg1 = fig.add_subplot(grid[1,0], projection=crs_sub)
gdf = pandas2geopandas(pd.concat([
    attrs[['lat', 'lon', 'markersize']],
    ds_timing_stats.sel(scen='tide').to_series().unstack().T
], axis=1))
circ_doy = gdf['mean']/365*2*np.pi
gdf['color'] = np.where(circ_doy>np.pi, -2*np.pi+circ_doy, circ_doy)
gdf0 = gdf[idx_sign==False].copy()
gdf1 = gdf[idx_sign].copy()
# main map
basemap(axg1, bbox=(-180, -60, 180, 90), gridlines=True, outline=True, **bmap_kwargs)
gdf0.plot(ax=axg1, marker='x', color=(0, 0, 0, 0.6), markersize=5, linewidth=0.2, alpha=0.6, legend=False)
gdf1.plot(ax=axg1, column='color', markersize=20, 
          norm=norm, vmin=amin, vmax=amax, cmap=cmap_phase,
          edgecolor=(0.5, 0.5, 0.5, 0.1), linewidth=0, legend=False)
axg1.text(-175, 75, letters[1], transform=crs_sub, fontsize=16)

# surge
axg2 = fig.add_subplot(grid[1,1], projection=crs_sub)
gdf = pandas2geopandas(pd.concat([
    attrs[['lat', 'lon', 'markersize']],
    ds_timing_stats.sel(scen='surge').to_series().unstack().T
], axis=1))
circ_doy = gdf['mean']/365*2*np.pi
gdf['color'] = np.where(circ_doy>np.pi, -2*np.pi+circ_doy, circ_doy)
gdf0 = gdf[idx_sign==False].copy()
gdf1 = gdf[idx_sign].copy()

# main map
basemap(axg2, bbox=(-180, -60, 180, 90), gridlines=True, outline=True, **bmap_kwargs)
gdf0.plot(ax=axg2, marker='x', color=(0, 0, 0, 0.6), markersize=5, linewidth=0.2, alpha=0.6, legend=False)
gdf1.plot(ax=axg2, column='color', markersize=20, 
          norm=norm, vmin=amin, vmax=amax, cmap=cmap_phase,
          edgecolor=(0.5, 0.5, 0.5, 0.1), linewidth=0, legend=False)
axg2.text(-175, 75, letters[2], transform=crs_sub, fontsize=16)

# add colorbar
posn = axg1.get_position() # fix colorbar position
cbar_ax = fig.add_axes([0, 0, 0.1, 0.1], polar=True) # new ax at random location.
cbar_ax.set_theta_offset(0.5*np.pi)
cbar_ax.set_theta_direction(-1)
cbar_ax.scatter(xval, yval, c=xval, s=45, cmap=cmap_phase, norm=norm, linewidths=0)
cbar_ax.set_yticks([])
cbar_ax.set_xticks(circ)
cbar_ax.set_xticklabels(dticks)
size = posn.height/4.
cbar_ax.set_position([posn.x0+size/4., posn.y0+posn.height/2-size*1.5, size, size])


fn_fig = join(fdir, f'timing_q{q}d{min_dist}_doy_mean.png')
print(basename(fn_fig))
plt.savefig(fn_fig)
# plt.close('all')